In [21]:
import datetime as dt
import pandas as pd
import pandas_datareader as web
import datetime as dt
import numpy as np
import matplotlib as plt

from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM


In [22]:
company = 'FB'
start = dt.datetime(2012,1,1)
end = dt.datetime(2020,1,1)

data = web.DataReader(company, 'yahoo', start, end)


In [23]:
#Prepare Data

scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(data['Close'].values.reshape(-1,1))

prediction_days = 60

x_train = []
y_train = []

for x in range(prediction_days, len(scaled_data)):
    x_train.append(scaled_data[x-prediction_days:x, 0 ])
    y_train.append(scaled_data[x,0])


x_train, y_train = np.array(x_train), np.array(y_train)

x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

In [26]:
# Build the Model

model = Sequential ()

model.add(LSTM(units=50, return_sequences=True, input_shape=(x_train.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50))
model.add(Dropout(0.2))
model.add(Dense(units=1))


model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(x_train, y_train, epochs=25, batch_size=32)



#TESTING THE MODEL


test_start = dt.datetime(2020,1,1)
test_end = dt.datetime(2021,1,1)

test_data = web.DataReader(company, 'yahoo', test_start,  test_end)
actual_prices = test_data['Close'].values

total_dataset = pd.concat((data['Close'], test_data['Close']), axis=0)


model_inputs = total_dataset[len(total_dataset) - len(test_data) - prediction_days:].values
model_inputs - model_inputs.reshape(-1, 1)
model_inputs = scaler.transform(model_inputs)

#Make predictions on test data

x_test = []

for x in range(prediction_days, len(model_inputs)):
    x_test.append(model_inputs[x-prediction_days:x, 0 ])


x_test = np.array(x_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

predicted_prices = model.predict(x_test)
predicted_prices = scaler.inverse_transform(predicted_prices)




#PLOT

plt.plot(actual_prices, color='black')
plt.plot(predicted_prices, color='green')

plt.legend()
plt.show()


Epoch 1/25
6/6 [==============================] - 3s 47ms/step - loss: 0.0716
Epoch 2/25
6/6 [==============================] - 0s 13ms/step - loss: 0.0254
Epoch 3/25
6/6 [==============================] - 0s 12ms/step - loss: 0.0231
Epoch 4/25
6/6 [==============================] - 0s 12ms/step - loss: 0.0126
Epoch 5/25
6/6 [==============================] - 0s 12ms/step - loss: 0.0143
Epoch 6/25
6/6 [==============================] - 0s 12ms/step - loss: 0.0105
Epoch 7/25
6/6 [==============================] - 0s 13ms/step - loss: 0.0100
Epoch 8/25
6/6 [==============================] - 0s 14ms/step - loss: 0.0091
Epoch 9/25
6/6 [==============================] - 0s 13ms/step - loss: 0.0105
Epoch 10/25
6/6 [==============================] - 0s 12ms/step - loss: 0.0096
Epoch 11/25
6/6 [==============================] - 0s 13ms/step - loss: 0.0087
Epoch 12/25
6/6 [==============================] - 0s 12ms/step - loss: 0.0105
Epoch 13/25
6/6 [==============================] - 0s 12ms/st

ValueError: Expected 2D array, got 1D array instead:
array=[170.88000488 162.88000488 163.27000427 163.49000549 158.05000305
 164.69999695 167.22999573 175.77999878 183.08999634 183.16999817
 169.27000427 166.6499939  159.1499939  169.58000183 160.72000122
 159.1000061  159.92999268 160.19000244 168.80000305 170.57000732
 167.11000061 170.25       168.52999878 178.33999634 177.49000549
 180.5        180.88999939 179.47000122 174.8500061  174.66000366
 167.96000671 163.05000305 161.11000061 163.25999451 168.77999878
 161.77999878 159.16999817 157.16000366 162.92999268 165.36000061
 160.32000732 158.53999329 160.38999939 162.05999756 169.1499939
 168.96000671 153.13000488 151.47000122 149.55000305 146.28999329
 148.02000427 146.08999634 142.11999512 142.82000732 140.41000366
 136.36999512 134.3999939  141.61000061 136.41000366 135.67999268
 343.01000977 326.23001099 332.95999146 333.64001465 329.22000122
 330.04998779 325.45001221 323.76998901 324.54000854 328.52999878
 324.76000977 335.